<a href="https://colab.research.google.com/github/safakan/CCTB_repo_safak_atakan_celik/blob/main/statistics/One_Way_Anova_and_Tuckey_Kramer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Comparing 2+ Group Means (One Way Anova + Tuckey-Kramer)

#### Helper Functions

In [ ]:
# Sum of Squares
def calculate_SST(grand_mean, observations_all):
  """
  """
  SST = 0
  for value in observations_all:
    SST += (value - grand_mean)**2
  return SST

def calculate_SSB(grand_mean, list_of_sample_means, list_of_sample_sizes):
  """
  """
  SSB = 0
  for i in range(len(list_of_sample_means)):
    SSB += list_of_sample_sizes[i] * (list_of_sample_means[i] - grand_mean)**2
  return SSB

def calculate_SSW(SST, SSB):
  """
  not implemented yet, almost like SST but use corresponding group means
  """
  return SST - SSB

# Degrees of Freedoms
def calculate_dfT(list_of_observation_counts):
  return sum(list_of_observation_counts) - 1

def calculate_dfB(group_count):
  return group_count - 1

def calculate_dfW(group_count, num_of_observations):
  return num_of_observations - group_count

# Mean Squares
## feel the redundancy
def calculate_MST(SST, dfT):
  return SST / dfT

def calculate_MSB(SSB, dfB):
  return SSB / dfB

def calculate_MSW(SSW, dfW):
  return SSW / dfW


## Tukey-Kramer Critical Range
def calculate_tukey_kramer_critical_range(q_critical, MSW, ni, nj):
    return q_critical * ( (MSW / 2) * ( (1 / ni) + (1 / nj))    )**0.5

#### Import Libraries and Load Data

In [ ]:
import scipy
import io
import pandas as pd

data = pd.read_csv(io.StringIO('''
Peter,Mike,Jason
2.5,2.1,2.8
2.6,2.4,3.7
2.7,2.2,3.3
2.2,2.9,3
2.55,2.6,
,3.1,
'''), header=0)

In [ ]:
data

,Peter,Mike,Jason
0,2.50,2.1,2.8
1,2.60,2.4,3.7
2,2.70,2.2,3.3
3,2.20,2.9,3.0
4,2.55,2.6,NaN
5,NaN,3.1,NaN


## Hypothesis Testing

### **One Way Anova**
1. State the null and alternative hypotheses
2. Choose the significance level (α)
3. Calculate means for each group and the grand mean
4. Calculate the test statistic (F-ratio)
5. Determine the critical F-value
6. Compare the test statistic to the critical F-value
7. If H0 is rejected, perform post-hoc tests to identify where the differences lie

In [ ]:
# 1. State the null and alternative hypotheses
H0 = "all means are equal"
Ha = "at least one mean is significantly different"

In [ ]:
# 2. Choose the significance level (α)
level_of_significance = 0.05

In [ ]:
# 3. Calculate means for each group and the grand mean
mean_peter = data['Peter'].mean()
mean_mike = data['Mike'].mean()
mean_jason = data['Jason'].mean()
grand_mean = (mean_peter + mean_mike + mean_jason) / 3

## 3.1 calculate counts
count_peter = data['Peter'].count()
count_mike = data['Mike'].count()
count_jason = data['Jason'].count()

## 3.2 store dps
dps = []
for col in data.columns:
    for value in data[col].dropna():
        dps.append(float(value))

In [ ]:
# 4. Calculate the test statistic (F-ratio)

SST = calculate_SST(grand_mean, dps)
SSB = calculate_SSB(grand_mean, [mean_peter, mean_mike, mean_jason], [count_peter, count_mike, count_jason])
SSW = SST - SSB

dfT = calculate_dfT([count_peter, count_mike, count_jason])
dfB = calculate_dfB(3) # group count - 1
dfW = dfT - dfB

MST = calculate_MST(SST, dfT)
MSB = calculate_MSB(SSB, dfB)
MSW = calculate_MSW(SSW, dfW)


F_statistic = MSB / MSW

print(f"""
    SST: {SST}
    SSB: {SSB}
    SSW: {SSW}

    dfT: {dfT}
    dfB: {dfB}
    dfW: {dfW}

    MST: {MST}
    MSB: {MSB}
    MSW: {MSW}

    F statistic: {F_statistic}
""")


    SST: 2.719166666666667
    SSB: 1.3421666666666678
    SSW: 1.3769999999999993

    dfT: 14
    dfB: 2
    dfW: 12

    MST: 0.19422619047619052
    MSB: 0.6710833333333339
    MSW: 0.11474999999999995

    F statistic: 5.848220769789405



In [ ]:
# 5. Determine the critical F-value
## needed: significance level, dfB (df1), dfW (df2)
## table look up: https://people.smp.uq.edu.au/YoniNazarathy/engineering_final_project_stats_material/Ftable.pdf 3.89
F_critical = scipy.stats.f.ppf(1-0.05, 2, 12)

# 5.1 Determine the p value (bonus step)
p_value = scipy.stats.f.sf(F_statistic, 2, 12)

In [ ]:
# 6. Compare the test statistic to the critical F-value
print(f"""
    F statistic: {F_statistic}
    F critical: {F_critical}

    P value: {p_value}
    Alpha: {level_of_significance}

    H0: {H0}
    Ha: {Ha}
""")

conditions= [(F_statistic > F_critical), (p_value < level_of_significance)]
if all(conditions):
    print("Reject H0")
else:
    print("Fail to reject H0")


    F statistic: 5.848220769789405
    F critical: 3.8852938346523933

    P value: 0.016865090464848947
    Alpha: 0.05

    H0: all means are equal
    Ha: at least one mean is significantly different

Reject H0


In [ ]:
# 7. If H0 is rejected, perform post-hoc tests to identify where the differences lie

## Tukey-Krumer - all possible pairwise comparisons while controlling the family-wise error rate

### **The Tukey-Kramer Multiple Comparison Test**
1. Calculate the critical value (𝑄𝛼) for the Tukey-Kramer test
2. Calculate the Tukey-Kramer critical range
3. Calculate the differences between the group means
4. Compare the differences between the group means to the Tukey-Kramer critical range

In [ ]:
# 1. Calculate the critical value (𝑄𝛼) for the Tukey-Kramer test
## table look up https://www.stat.purdue.edu/%7Elingsong/teaching/2018fall/q-table.pdf
## alpha = 0.05, k = 3 dfw = 9, Qalpha = 3.948

level_of_significance
number_of_groups = 3
dfW

from scipy.stats import studentized_range
# To get the critical q-value:
q_critical = studentized_range.ppf(q=1-level_of_significance, k=number_of_groups, df=dfW)

In [ ]:
# 2. Calculate the Tukey-Kramer critical range for each pair
# 3. Calculate the differences between the group means
# 4. Compare the differences between the group means to the Tukey-Kramer critical range

In [ ]:
# Create a dictionary for ease of use
means = {
    'Peter': mean_peter,
    'Mike': mean_mike,
    'Jason': mean_jason
}

# group names for ease of use
groups = list(means.keys())

# start a loop to compare each pair
for i in range(len(groups)):
    for j in range(i + 1, len(groups)):
        group1 = groups[i]
        group2 = groups[j]

        n1 = data[group1].count()
        n2 = data[group2].count()

        critical_range = calculate_tukey_kramer_critical_range(q_critical, MSW, n1, n2)
        mean_diff = abs(means[group1] - means[group2])

        print(f"Comparison: {group1} vs {group2}")
        print(f"  Mean Difference: {mean_diff}")
        print(f"  Critical Range: {critical_range}")
        if mean_diff > critical_range:
            print(f"  The difference between {group1} and {group2} is significant.")
        else:
            print(f"  The difference between {group1} and {group2} is not significant.")
        print("---")

Comparison: Peter vs Mike
  Mean Difference: 0.03999999999999959
  Critical Range: 0.5472373021145567
  The difference between Peter and Mike is not significant.
---
Comparison: Peter vs Jason
  Mean Difference: 0.69
  Critical Range: 0.6062422989310949
  The difference between Peter and Jason is significant.
---
Comparison: Mike vs Jason
  Mean Difference: 0.6500000000000004
  Critical Range: 0.5833569241366753
  The difference between Mike and Jason is significant.
---


---
---

#### Notes

- F-test statistic follows an F distribution, right skewed

- MST = MSB + MSW is not a valid equation unlike others. This is due to denominators.
 - MSB = SSB / (k-1), where k is number of groups
 - MSW = SSW / (N-k), where N is total sample size
 - MST = SST / (N-1)